In [1]:
!pip install langchain
!pip install langchain-community
!pip install langchain-experimental
!pip install google-generativeai
!pip install langchain-google-genai
!pip install pypdf
!pip install unstructured
!pip install openpyxl
!pip install docx2txt
!pip insttall python-magic
!pip install python-pptx
!pip install faiss-cpu
!pip install tiktoken
!pip install -U langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 17.5 MB/s eta 0:00

In [50]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from langchain_community.vectorstores.faiss import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import RetrievalQA
from langchain_core.runnables import RunnablePassthrough
from langchain import hub
from langchain.chains import LLMChain, StuffDocumentsChain

In [43]:
loader1=PyPDFLoader('pdf1.pdf')
loader2=PyPDFLoader('pdf2.pdf')
document1=loader1.load()
document2=loader2.load()
print(document1)
print(document2)

[Document(metadata={'source': 'pdf1.pdf', 'page': 0}, page_content='This is blue dogs ')]
[Document(metadata={'source': 'pdf2.pdf', 'page': 0}, page_content='This is chocolate dogs ')]


In [59]:
textsplitter=RecursiveCharacterTextSplitter(chunk_size=20,chunk_overlap=10)
chunks1=textsplitter.split_documents(document1)
chunks2=textsplitter.split_documents(document2)
chunks=chunks1 + chunks2
print(chunks)

[Document(metadata={'source': 'pdf1.pdf', 'page': 0}, page_content='This is blue dogs'), Document(metadata={'source': 'pdf2.pdf', 'page': 0}, page_content='This is chocolate'), Document(metadata={'source': 'pdf2.pdf', 'page': 0}, page_content='chocolate dogs')]


In [ ]:
os.environ['GOOGLE_API_KEY']='AIzaSyBxpx1h'
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector_db=FAISS.from_documents(chunks,embeddings)
retriever=vector_db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'score_threshold': 0.4})

In [63]:
query='what the color of dogs in these uploaded documents'
similarity=vector_db.similarity_search(query)
for chunk in similarity:
    print(chunk)

page_content='This is blue dogs' metadata={'source': 'pdf1.pdf', 'page': 0}
page_content='chocolate dogs' metadata={'source': 'pdf2.pdf', 'page': 0}
page_content='This is chocolate' metadata={'source': 'pdf2.pdf', 'page': 0}


In [ ]:
os.environ['GOOGLE_API_KEY']='AIzaSyBxp'
llm=ChatGoogleGenerativeAI(model='gemini-1.5-pro')
prompt=hub.pull('rlm/rag-prompt')
qa_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)
qa_chain.invoke(query)

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


'The documents mention blue and chocolate-colored dogs.  Therefore, the dogs are blue and chocolate colored.  \n'